# 创建你的第一个 Secure Aggregation FedAvg 联邦学习任务

Secure Aggregation FedAvg 算法（以下简称 SecureFedAvg 算法）是 FedAvg 算法的一个扩展，在原算法基础上提供了隐私参数的支持。如果你对 FedAvg 算法及其使用方式还不了解，请先移步至[创建你的第一个 FedAvg 联邦学习任务](FedAvg.ipynb)，然后再回来继续阅读。

SecureFedAvg 算法是 FedAvg 算法的一个扩展，因此二者的大部分内容是相同的，所以这里只介绍二者存在差异的部分。

### 初始化参数

SecureFedAvg 需要一个额外的超参数 t，用于控制秘密共享时恢复秘密所需的最少秘密切片数量。以下代码展示了如何定义一个 SecureFedAvg 算法的实现。

In [ ]:
import os
from time import time
from typing import Dict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch import optim
from torch.utils.data import DataLoader

from alphafed import logger
from alphafed.fed_avg import SecureFedAvgScheduler

class DemoSecureFedAvg(SecureFedAvgScheduler):
    ...


scheduler = DemoSecureFedAvg(t=2,
                             max_rounds=5,
                             log_rounds=1,
                             calculation_timeout=120)

SecureFedAvg 算法其余部分的实现方式和要求与 FedAvg 算法完全一致，请参考 FedAvg 算法部分的说明。下面是一个完整的 SecureFedAvg 算法示例：

In [ ]:
import os
from time import time
from typing import Dict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch import optim
from torch.utils.data import DataLoader

from alphafed import get_dataset_dir, logger
from alphafed.fed_avg import SecureFedAvgScheduler


class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features=320, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)


class DemoSecureFedAvg(SecureFedAvgScheduler):

    def __init__(self,
                 t: int,
                 max_rounds: int = 0,
                 merge_epoch: int = 1,
                 calculation_timeout: int = 300,
                 log_rounds: int = 0) -> None:
        super().__init__(t=t,
                         max_rounds=max_rounds,
                         merge_epochs=merge_epoch,
                         calculation_timeout=calculation_timeout,
                         log_rounds=log_rounds)
        self.batch_size = 64
        self.learning_rate = 0.01
        self.momentum = 0.5
        self.log_interval = 5
        self.random_seed = 42

        torch.manual_seed(self.random_seed)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def build_model(self) -> nn.Module:
        model = ConvNet()
        return model.to(self.device)

    def build_optimizer(self, model: nn.Module) -> optim.Optimizer:
        return optim.SGD(model.parameters(),
                        lr=self.learning_rate,
                        momentum=self.momentum)

    def build_train_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                get_dataset_dir(self.task_id),
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=True
        )

    def build_test_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                get_dataset_dir(self.task_id),
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=False
        )

    def state_dict(self) -> Dict[str, torch.Tensor]:
        return self.model.state_dict()

    def load_state_dict(self, state_dict: Dict[str, torch.Tensor]):
        self.model.load_state_dict(state_dict)

    def validate_context(self):
        super().validate_context()
        assert self.train_loader and len(self.train_loader) > 0, 'failed to load train data'
        self.push_log(f'There are {len(self.train_loader.dataset)} samples for training.')
        assert self.test_loader and len(self.test_loader) > 0, 'failed to load test data'
        self.push_log(f'There are {len(self.test_loader.dataset)} samples for testing.')

    def train_an_epoch(self) -> None:
        self.model.train()
        for data, labels in self.train_loader:
            data: torch.Tensor
            labels: torch.Tensor
            data, labels = data.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = F.nll_loss(output, labels)
            loss.backward()
            self.optimizer.step()

    def run_test(self):
        self.model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, labels in self.test_loader:
                data: torch.Tensor
                labels: torch.Tensor
                data, labels = data.to(self.device), labels.to(self.device)
                output: torch.Tensor = self.model(data)
                test_loss += F.nll_loss(output, labels, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                correct += pred.eq(labels.view_as(pred)).sum().item()

        test_loss /= len(self.test_loader.dataset)
        correct_rate = 100. * correct / len(self.test_loader.dataset)
        logger.info(f'Test set: Average loss: {test_loss:.4f}')
        logger.info(
            f'Test set: Accuracy: {correct}/{len(self.test_loader.dataset)} ({correct_rate:.2f}%)'
        )


scheduler = DemoSecureFedAvg(t=2,
                             max_rounds=5,
                             log_rounds=1,
                             calculation_timeout=120)
scheduler.submit(task_id='YOUR_TASK_ID')